# The weather forecast for today

In [1]:
import datetime as dt 
import requests
import json
import pandas as pd 

pd.options.display.max_rows = 500

/Users/mikkelpedersen/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
base_url = 'https://api.openweathermap.org/data/2.5/weather?'
API_key = 'b765e9524535b7f51e9fbc2cce7f076d'

### List of cities

In [3]:
city_list = pd.read_csv('/Users/mikkelpedersen/Desktop/project_vs_studio/wheater_api/cities_list.csv', delimiter=';')

In [4]:
city_list.head(20)

,name,lon,lat,country
0,Abidjan,"-4,00167","5,35444",Ivory Coast
1,Abu Dhabi,"54,39696","24,45118",United Arab Emirates
2,Abuja,"7,49508","9,05785",Nigeria
3,Accra,"-0,1969","5,55602",Ghana
4,Addis Ababa,"38,74689","9,02497",Ethiopia
5,Ahmedabad,"72,58727","23,02579",India
6,Aleppo,"37,16117","36,20124",Syria
7,Alexandria,"29,91582","31,20176",Egypt
8,Algiers,"3,08746","36,73225",Algeria
9,Almaty,"76,92861","43,25667",Kazakhstan


In [7]:
def kelvin_to_celcius_fahrenhit(kelvin):
    celcius = kelvin - 273
    fahrenheit = celcius * (9/5) + 32
    return celcius, fahrenheit

In [8]:
def see_forecast(city_name):
    url_forecast = base_url + 'appid=' + API_key + '&q=' + city_name
    wforecast = requests.get(url_forecast).json()  
    
    wfcity_name = wforecast['name']
    wflat_city = wforecast['coord']['lat']
    wflon_city = wforecast['coord']['lon']
    wfcountry = wforecast['sys']['country']

    wftemp_kelvin = wforecast['main']['temp']
    wftemp_celcius, wftemp_fahrenhit = kelvin_to_celcius_fahrenhit(wftemp_kelvin)
    wffeels_like = wforecast['main']['feels_like']
    wffeels_like_celsius, wffeels_like_fahrenhit = kelvin_to_celcius_fahrenhit(wffeels_like)
    wfhumidity = wforecast['main']['humidity']
    wfwindspeed = wforecast['wind']['speed']
    wfdescription = wforecast['weather'][0]['description']
    wfclouds = wforecast['clouds']['all']

    wfsunrise_time = dt.datetime.utcfromtimestamp(wforecast['sys']['sunrise'] + wforecast['timezone'])
    wfsunset_time = dt.datetime.utcfromtimestamp(wforecast['sys']['sunset'] + wforecast['timezone'])
    
    
    print(f'This is the city of which you are seeing the weather forecast: {wfcity_name}')
    print(f'This is the longtitude: {wflon_city} and latitude: {wflat_city} of the city')
    print(f'this is the county of the city: {wfcountry}')
    print(f'the temp is {round(wftemp_celcius,2)} in celsius, and {round(wftemp_fahrenhit, 2)} fahrenhit')
    print(f'The tempature feels like {round(wffeels_like_celsius,2)} in celsius and {round(wffeels_like_fahrenhit,2)} in fahrenhit')
    print(f'The humidity is: {wfhumidity}')
    print(f'The windspeed is: {wfwindspeed}')
    print(f'Today the weather is {wfdescription}')
    print(f'The amount of clouds is: {wfclouds}')
    print(f'Today the sun will rise at {wfsunrise_time} and set at {wfsunset_time}')


In [9]:
see_forecast('Copenhagen')

This is the city of which you are seeing the weather forecast: Copenhagen
This is the longtitude: 12.5655 and latitude: 55.6759 of the city
this is the county of the city: DK
the temp is 8.79 in celsius, and 47.82 fahrenhit
The tempature feels like 4.86 in celsius and 40.75 in fahrenhit
The humidity is: 80
The windspeed is: 8.75
Today the weather is clear sky
The amount of clouds is: 2
Today the sun will rise at 2024-10-03 07:16:50 and set at 2024-10-03 18:40:19


### Map the forecast into a pandas DataFrame

In [5]:
city = 'Frederiksberg'
url = base_url + 'appid=' + API_key + '&q=' + city
response = requests.get(url).json()  

In [6]:
print(response)

{'coord': {'lon': 12.5346, 'lat': 55.6794}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 281.39, 'feels_like': 277.21, 'temp_min': 279.67, 'temp_max': 282.49, 'pressure': 1018, 'humidity': 83, 'sea_level': 1018, 'grnd_level': 1017}, 'visibility': 10000, 'wind': {'speed': 9.26, 'deg': 30}, 'clouds': {'all': 23}, 'dt': 1727935950, 'sys': {'type': 2, 'id': 2035645, 'country': 'DK', 'sunrise': 1727932617, 'sunset': 1727973627}, 'timezone': 7200, 'id': 2621942, 'name': 'Frederiksberg', 'cod': 200}


In [18]:
existing_df = pd.read_csv('weather_data.csv')

In [19]:
def flatten_dict(d, parent_key='', sep='_'):
    """
    Flatten a nested dictionary.
    
    Args:
        d (dict): The dictionary to flatten.
        parent_key (str): The base key string for nested keys.
        sep (str): Separator to use between parent and child keys.
        
    Returns:
        dict: A flattened dictionary.
    """
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        elif isinstance(v, list):
            for i, item in enumerate(v):
                if isinstance(item, dict):
                    items.update(flatten_dict(item, f"{new_key}{sep}{i}", sep=sep))
                else:
                    items[f"{new_key}{sep}{i}"] = item
        else:
            items[new_key] = v
    return items

In [20]:
print(response)

{'coord': {'lon': 12.5346, 'lat': 55.6794}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 281.39, 'feels_like': 277.21, 'temp_min': 279.67, 'temp_max': 282.49, 'pressure': 1018, 'humidity': 83, 'sea_level': 1018, 'grnd_level': 1017}, 'visibility': 10000, 'wind': {'speed': 9.26, 'deg': 30}, 'clouds': {'all': 23}, 'dt': 1727935950, 'sys': {'type': 2, 'id': 2035645, 'country': 'DK', 'sunrise': 1727932617, 'sunset': 1727973627}, 'timezone': 7200, 'id': 2621942, 'name': 'Frederiksberg', 'cod': 200}


In [22]:
temp_kelvin = response['main']['temp']
temp_celcius, temp_fahrenhit = kelvin_to_celcius_fahrenhit(temp_kelvin)
feels_like = response['main']['feels_like']
feels_like_celsius, feels_like_fahrenhit = kelvin_to_celcius_fahrenhit(feels_like)



#'gust': response['wind']['gust']  -> if the api does not provide it
data = {
    'coord': {'lon': response['coord']['lon'], 'lat': response['coord']['lat']},
    'weather': [{'id': response['weather'][0]['id'],
                 'main': response['weather'][0]['main'],
                 'description': response['weather'][0]['description'], 
                 'icon': response['weather'][0]['icon']}],
    'base': response['base'],
    'main': {
        'temp': response['main']['temp'],
        'temp_celsius': temp_celcius,
        'feel_like_celsius': feels_like_celsius,
        'feels_like': response['main']['feels_like'],
        'temp_min': response['main']['temp_min'],
        'temp_max': response['main']['temp_max'],
        'pressure': response['main']['pressure'],
        'humidity': response['main']['humidity'],
        'sea_level': response['main']['sea_level'],
        'grnd_level': response['main']['grnd_level']
    },
    'visibility': response['visibility'],
    'wind': {'speed': response['wind']['speed'], 'deg': response['wind']['deg']},
    'clouds': {'all': response['clouds']['all']},
    'dt': response['dt'],
    'sys': {
        'type': response['sys']['type'],
        'id': response['sys']['id'],
        'country': response['sys']['country'],
        'sunrise': response['sys']['sunrise'],
        'sunset': response['sys']['sunset']
    },
    'timezone': response['timezone'],
    'id': response['id'],
    'name': response['name'],
    'cod': response['cod']
}

flattened_data = flatten_dict(data)


In [23]:
new_df = pd.concat([existing_df, pd.DataFrame([flattened_data])], ignore_index=True)

# Step 3: Concatenate the new DataFrame with the existing DataFrame
#updated_df = pd.concat([existing_df, new_df], ignore_index=True)

# Step 4: Save the updated DataFrame back to the CSV file
new_df.to_csv('weather_data.csv', index=False)